In [1]:
from pyproj import Transformer

# Initialisieren der Transformer
wgs_to_utm_transformer = Transformer.from_crs("epsg:4326", "epsg:25832", always_xy=True)
utm_to_wgs_transformer = Transformer.from_crs("epsg:25832", "epsg:4326", always_xy=True)

# Verwenden der definierten Funktion, um die Transformation für die angegebenen Koordinaten durchzuführen:
# 48,131507 (Breitengrad), 11,689532 (Längengrad)

# Definierte Funktion zur Transformation
def transform_lat_lon_to_utm(lat, lon):
    """
    Transformiert geografische Koordinaten in ETRS89 / UTM Zone 32N Koordinaten.
    
    :param lat: Breitengrad
    :param lon: Längengrad
    :return: Tuple (x, y) repräsentiert die transformierten UTM Koordinaten (Easting, Northing)
    """
    transformer = Transformer.from_crs("epsg:4326", "epsg:25832", always_xy=True)
    x, y = transformer.transform(lon, lat)
    return x, y

# Koordinaten für die Transformation
lat_input, lon_input = 48.131507, 11.689532

# Durchführung der Transformation
transform_lat_lon_to_utm(lat_input, lon_input)

(700108.6343251554, 5334415.579024226)

In [8]:
import geopandas as gpd
from shapely.geometry import Point

def check_location_inside(file_path, location):
    # Load the file containing the Bezirks polygons
    bezirks = gpd.read_file(file_path)
    
    # Convert the location to a Point geometry
    point = Point(location)
    
    # Check if the point is inside any of the Bezirks polygons
    for index, row in bezirks.iterrows():
        if point.within(row['geometry']):
            return row['sb_nummer']  # Return the ID of the Bezirk if the location is inside
    
    return None  # Return None if the location is not inside any Bezirk

# Example usage:
file_path = "/Users/janis/Desktop/Muc_electro_vehicles/data/muc_geojson.geojson"  # Replace with the actual path to your GeoJSON file
location = (700108.6343251554, 5334415.579024226)
bezirk_id = check_location_inside(file_path, location)
if bezirk_id is not None:
    print(f"The location is inside Bezirk {bezirk_id}.")
else:
    print("The location is not inside any Bezirk.")

The location is inside Bezirk 15.


In [21]:
from pyproj import Transformer
import geopandas as gpd
from shapely.geometry import Point

# Define the transformer for converting latitude and longitude to UTM
transformer = Transformer.from_crs("epsg:4326", "epsg:25832", always_xy=True)

def transform_lat_lon_to_utm(lat, lon):
    """
    Transform latitude and longitude to ETRS89 / UTM Zone 32N coordinates.

    :param lat: Latitude
    :param lon: Longitude
    :return: Tuple (x, y) representing the transformed UTM coordinates (Easting, Northing)
    """
    x, y = transformer.transform(lon, lat)
    return x, y

def check_location_inside(file_path, lat, lon):
    """
    Checks if the given geographic location (latitude and longitude) is inside any Stadtbezirk.

    :param file_path: Path to the GeoJSON file containing Stadtbezirk polygons
    :param lat: Latitude of the location
    :param lon: Longitude of the location
    :return: The ID of the Stadtbezirk if inside, otherwise None
    """
    # Load the GeoJSON file with Stadtbezirk polygons
    bezirks = gpd.read_file(file_path)

    # Transform latitude and longitude to UTM coordinates
    x, y = transform_lat_lon_to_utm(lat, lon)

    # Convert the UTM coordinates to a Point geometry
    point = Point(x, y)

    # Check if the point is inside any of the Stadtbezirk polygons
    for _, row in bezirks.iterrows():
        if point.within(row['geometry']):
            return row['sb_nummer']  # Return the ID of the Stadtbezirk

    return None  # Return None if the location is not inside any Stadtbezirk

# Example usage:
file_path = "/Users/janis/Desktop/Muc_electro_vehicles/data/muc_geojson.geojson"  # Update with the correct path
lat, lon = 48.131507, 11.689532  # Sample coordinates
bezirk_id = check_location_inside(file_path, lat, lon)
if bezirk_id:
    print(f"The location is inside Bezirk {bezirk_id}.")
else:
    print("Not in MUC.")


The location is inside Bezirk 15.


In [ ]:
import pandas as pd

# Load the CSV file into a DataFrame
file_path_csv = '/Users/janis/Desktop/Muc_electro_vehicles/data/ladestationen_muc_janis.csv'
df = pd.read_csv(file_path_csv)

# Assume your GeoJSON file path is correct and accessible
geojson_file_path = '/Users/janis/Desktop/Muc_electro_vehicles/data/muc_geojson.geojson'  # Update this path

# Apply the transformation and check function to each row in the DataFrame
# Here, we assume that the columns in your CSV are named 'Längengrad' and 'Breitengrad'
# Adjust column names if they are different
df['Stadtbezirk'] = df.apply(lambda row: check_location_inside(geojson_file_path, row['Breitengrad'], row['Längengrad']), axis=1)

df.head()
# Save the modified DataFrame back to a new CSV file
#output_file_path = 'data/Ladesaeulenregister_with_Stadtbezirk.csv'
#df.to_csv(output_file_path, index=False)

In [19]:
import pandas as pd

# Load the CSV file into a DataFrame
file_path_csv = '/Users/janis/Desktop/Muc_electro_vehicles/data/ladestationen_muc_janis.csv'
df = pd.read_csv(file_path_csv)
df.head()
# Define the path to the GeoJSON file
#geojson_file_path = '/Users/janis/Desktop/Muc_electro_vehicles/data/muc_geojson.geojson'  # Update this path

# Ensure that the lambda function within apply correctly references scalar values for each row
#df['Stadtbezirk'] = df.apply(lambda row: check_location_inside(geojson_file_path, row['Breitengrad'], row['Längengrad']), axis=1)

# Save the modified DataFrame to a new CSV file
#output_file_path = '/mnt/data/Ladesaeulenregister_with_Stadtbezirk.csv'
#df.to_csv(output_file_path, index=False)


,Betreiber,Straße,Hausnummer,Adresszusatz,Postleitzahl,Ort,Bundesland,Kreis/kreisfreie Stadt,Breitengrad,Längengrad,...,Public Key1,Steckertypen2,P2 [kW],Public Key2,Steckertypen3,P3 [kW],Public Key3,Steckertypen4,P4 [kW],Public Key4
0,SWM Versorgungs GmbH,Karl-Hromadnik-Str.,2,NaN,80241,München,Bayern,Kreisfreie Stadt München,"48,137018","11,456796",...,NaN,AC Steckdose Typ 2,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SWM Versorgungs GmbH,Peter-Kreuder-Str.,5,NaN,80245,München,Bayern,Kreisfreie Stadt München,"48,159746","11,445709",...,NaN,AC Steckdose Typ 2,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SWM Versorgungs GmbH,Peter-Kreuder-Str.,5a,NaN,80245,München,Bayern,Kreisfreie Stadt München,"48,159701","11,445643",...,NaN,AC Steckdose Typ 2,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SWM Versorgungs GmbH,Wiesentfelser Str.,16,NaN,80249,München,Bayern,Kreisfreie Stadt München,"48,147492","11,419531",...,NaN,AC Steckdose Typ 2,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Stadtsparkasse München AdöR,Sparkassen Str.,2-4,NaN,80331,München,Bayern,Kreisfreie Stadt München,"48,1368071","11,5778407",...,3059301306072a8648ce3d020106082a8648ce3d030107...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
import pandas as pd

# Load the CSV file into a DataFrame
file_path_csv = '/Users/janis/Desktop/Muc_electro_vehicles/data/ladestationen_muc_janis.csv'
df = pd.read_csv(file_path_csv)

# Replace ',' with '.' in the 'Längengrad' and 'Breitengrad' columns and convert them to floats
df['Längengrad'] = df['Längengrad'].str.replace(',', '.').astype(float)
df['Breitengrad'] = df['Breitengrad'].str.replace(',', '.').astype(float)

# Define the path to the GeoJSON file
geojson_file_path = '/Users/janis/Desktop/Muc_electro_vehicles/data/muc_geojson.geojson'  # Update this path

# Apply the transformation and check function to each row in the DataFrame
df['Stadtbezirk'] = df.apply(lambda row: check_location_inside(geojson_file_path, row['Breitengrad'], row['Längengrad']), axis=1)

# Save the modified DataFrame to a new CSV file
output_file_path = 'Ladesaeulenregister_with_Stadtbezirk.csv'
df.to_csv(output_file_path, index=False)
